In [1]:
from IPython.display import clear_output, display, HTML
import time
import pandas as pd
# -------------
# 데모 버전
# -------------

# 0. git에 저장된 대시보드용 파일 가져와서 실행 시키기

url = 'https://raw.githubusercontent.com/bories97/Python/refs/heads/main/%EA%B3%B5%EB%AA%A8%EC%A0%84%20%EB%8D%B0%EC%9D%B4%ED%84%B0/match_info.csv'
url1 = 'https://raw.githubusercontent.com/bories97/Python/refs/heads/main/%EA%B3%B5%EB%AA%A8%EC%A0%84%20%EB%8D%B0%EC%9D%B4%ED%84%B0/state_df.csv'
url2 =  'https://raw.githubusercontent.com/bories97/Python/refs/heads/main/%EA%B3%B5%EB%AA%A8%EC%A0%84%20%EB%8D%B0%EC%9D%B4%ED%84%B0/demo_df.csv'

match = pd.read_csv(url)
states = pd.read_csv(url1)
demo = pd.read_csv(url2)

# 1. 설정
available_games = (match[['game_id', 'home_team_name_ko', 'away_team_name_ko']].drop_duplicates().sort_values('game_id'))

print("\n선택 가능한 경기 목록:\n")

for _, row in available_games.iterrows():
    print(f"[{row['game_id']}] {row['home_team_name_ko']} vs {row['away_team_name_ko']}")

print("\n")
SELECTED_GAME_ID = int(input("경기 ID를 입력하세요"))   # 원하는 경기 ID
REPLAY_SPEED = 0.2        # 초 단위 재생 속도


# 2. 시각화용 데이터 준비 (state_df 기준)

viz_df = (states[states['game_id'] == SELECTED_GAME_ID].sort_values('time_bin').reset_index(drop=True))

home_name = match.loc[match['game_id'] == SELECTED_GAME_ID, 'home_team_name_ko'].values[0]

away_name = match.loc[match['game_id'] == SELECTED_GAME_ID, 'away_team_name_ko'].values[0]

print(f"경기 분석 시작: {home_name} vs {away_name}")
time.sleep(1)


# 3. 리플레이 루프

for i in range(len(viz_df)):
    row = viz_df.iloc[i]
    clear_output(wait=True)


    # 1. 현재 시점

    current_time_sec = row['time_bin']


    # 2. 해당 시점의 실제 스코어 (soccer_df 기준)

    score_row = (demo[(demo['game_id'] == SELECTED_GAME_ID) &
                          (demo['time_bin'] <= current_time_sec)].sort_values('time_bin'))
    if len(score_row) > 0:
        last_score = score_row.iloc[-1]
        h_score = int(last_score['home_score'])
        a_score = int(last_score['away_score'])
    else:
        h_score, a_score = 0, 0


    # 3. 승/무/패 확률

    p_h = row['p_home'] * 100
    p_d = row['p_draw'] * 100
    p_a = row['p_away'] * 100


    # 4. 공격 모멘텀 계산 (앞에서 정의한 가중치 그대로)

    momentum = (0.15 * row['home_xgdiff'] + 0.10 * row['home_xadiff'] + 0.05 * row['home_xtdiff'])


    # 5. 해설 문구 생성

    if abs(momentum) > 0.02:
        team = home_name if momentum > 0 else away_name
        color = "#e03e2d" if momentum > 0 else "#2d6ae0"
        commentary = (f"<span style='color:{color}; font-weight:bold;'>"
                      f" {team} 공격 흐름 우세 </span>")
    elif row['remain_time'] < 600 and row['score_diff'] != 0:
        commentary = ("<span style='color:#555; font-weight:bold;'>"" 경기 막판 굳히기 국면</span>")
    else:
        commentary = ("<span style='color:#aaa;'> 탐색전 진행 중</span>")


    # 6. HTML 출력

    html_code = f"""
    <div style="
        width: 600px; margin: 0 auto; font-family: Arial, sans-serif;
        border: 1px solid #e1e4e8; border-radius: 12px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        background-color: white;
    ">
        <div style="
            background-color: #f6f8fa; padding: 10px;
            text-align: center; font-size: 14px; font-weight: 600;
        ">
            ⏱ Match Time: {int(row['time_bin'])} min
        </div>

        <div style="padding: 25px 0; display: flex; justify-content: space-around;">
            <div style="width: 35%; text-align: center; font-size: 18px; font-weight: bold;">
                {home_name}
            </div>
            <div style="width: 30%; text-align: center; font-size: 40px; font-weight: 800;">
                <span style="color:#e03e2d;">{h_score}</span>
                <span style="font-size:24px; color:#ccc;"> : </span>
                <span style="color:#2d6ae0;">{a_score}</span>
            </div>
            <div style="width: 35%; text-align: center; font-size: 18px; font-weight: bold;">
                {away_name}
            </div>
        </div>

        <div style="padding: 0 30px 25px 30px;">
            <div style="display: flex; justify-content: space-between; font-size: 12px;">
                <span> Home {p_h:.1f}%</span>
                <span> Draw {p_d:.1f}%</span>
                <span> Away {p_a:.1f}%</span>
            </div>
            <div style="display: flex; height: 10px; background-color: #eee; border-radius: 5px;">
                <div style="width:{p_h}%; background:#e03e2d;"></div>
                <div style="width:{p_d}%; background:#999;"></div>
                <div style="width:{p_a}%; background:#2d6ae0;"></div>
            </div>
        </div>

        <div style="
            background-color:#f6f8fa; padding:12px;
            text-align:center; font-size:14px;
        ">
            {commentary}
        </div>
    </div>
    """

    display(HTML(html_code))
    time.sleep(REPLAY_SPEED)

print("경기 리플레이 종료")

경기 리플레이 종료
